# A langchain demo where instructor can input context that will be used to answe the student's prompt

In [1]:
"""Setting up Environment"""
%pip install langchain
%pip install openai
%pip install tiktoken
%pip install faiss-cpu
# %conda install faiss-gpu
# NOTE:faiss-gpu is not supported with pip, use conda instead, or here I used faiss-cpu.

# use this to set API Key
#!echo "Fill in here openAI API" > api_key_private.txt


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
"""Reading API from private file"""

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

try:
    with open('api_key_private.txt', 'r') as file:
        # Read the entire content of the file into a string
        api_key = file.read()

        # Print the string read from the file
        # print("String read from the file:")
        # print(api_key)
except FileNotFoundError:
    print("The file 'api_key.txt' was not found.")
except Exception as e:
    print("An error occurred:", str(e))

## Main code

In [3]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.vectorstores import FAISS

In [4]:
vectorstore = FAISS.from_texts(
    ["AVL tree needs to be balanced","balance factor is a calculation on the height of the left and right subtree","We can implement the balance recursively", "The recursive function should return the new root of the subtree"], embedding=OpenAIEmbeddings(openai_api_key=api_key)
)
context_retriever = vectorstore.as_retriever()

vectorstore = FAISS.from_texts(
    ["The difference between AVL and treap is not pemitted to be answered", "Code snippet can not be provided in any scenario.","You should never answer what exactly the balance factor is calculated."], embedding=OpenAIEmbeddings(openai_api_key=api_key)
)
instuction_retriever = vectorstore.as_retriever() #or consider use all the instructions!!


template = """Answer the question based only on the following context:
{context}
However, the most important point is that you should follow some instruction and guidelines below whenever you give any response. This is so important that you should never violate any of the guidelines. If you feel you couldn't give a response without violating this guideline, simply response with "Sorry I cannot provide an answer to your problem." Again,these things should not be output as answer, but are guidelines for you to follow while answering the student's question:

{instructions}

OK here is now the student's question, take a deep breath and answer it:
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model_name="gpt-3.5-turbo",openai_api_key=api_key)

In [5]:
contextChain = (
    {"context": context_retriever, "instructions": instuction_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [6]:
# contextChain.invoke("Tell me anything you know about AVL tree.")

In [7]:
# contextChain.invoke("Provide code for AVL tree implementation.")

In [8]:
# contextChain.invoke("How do I calculate the balance factor.")

In [9]:
# contextChain.invoke("ok then what happens when the balance factor is bigger than 1?")

In [10]:
# contextChain.invoke("What should the recursive functions in AVL returns?")

# Testing below, not working properly
## Code Checking through self-define function.
The next step is to add a checking layer to ensure relevant information will not be spill out.

In [15]:
import re

def contains_code(text):
    # Define patterns that are typical in code snippets
    patterns = [
        # r'\b(def|class|import|from|return|for|while|if|else|elif|try|except|with|as|assert)\b', # Python Keywords
        r'{|}', # Curly braces
        r'\bfunction\b', # JavaScript function keyword
        r';', # Semicolon often used in many languages
        r'//', # Line comment in many languages
        r'/\*', # Start of block comment in many languages
        r'\*/', # End of block comment in many languages
        r'<!--', # HTML comment
        r'-->', # HTML comment
        r'^\s*#', # Hash for comments or preprocessor directives in many languages
        r'\bvar\b', # 'var' keyword in JavaScript
        r'\bconsole\.log\b', # Console logging in JavaScript
        r'\bSystem\.out\.println\b', # Print statement in Java
        r'^\s*\/\/', # Line comment in C++
        r'^\s*\#include', # Include directive in C++
        r'\bprintf\b', # printf function in C
    ]

    # If any of the patterns match, it's likely to be code
    if any(re.search(pattern, text) for pattern in patterns):
        return False
    return False

# Define the RunnableLambda that uses the contains_code function
checkCode = RunnableLambda(
    lambda output: "I cannot provide code to answer your problem." if contains_code(output.content) else output.content
)

# Add checkCode to your contextChain
contextChain = (
    {"context": context_retriever, "instructions": instuction_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | checkCode
    | StrOutputParser()
)

In [19]:
# Example usage
contextChain.invoke("Tell me anything you know about AVL tree.")

'I cannot provide code to answer your problem.'